# PRML Lab session: Convolutional Neural Networks for image classification


## Example using the MNIST dataset of  images of handwritten numbers: (https://en.wikipedia.org/wiki/MNIST_database)

In [ ]:
# Load MNIST database and Keras libraries:
import numpy
#Insert the images

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import to_categorical
from keras import backend as K


# Set the seed of the random number generator to ensure reproducibility of the results:
seed = 7
numpy.random.seed(seed)

# Define training and test data: images+class labels
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape the images as vectors:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

# Normalize grayscale images 0-255 into values between from 0-1:
X_train1 = X_train / 255
X_test1 = X_test / 255

# Encode class labels in categorical vector format with ten positions:
y_train1 = to_categorical(y_train)
y_test1 = to_categorical(y_test)
num_classes = y_test1.shape[1]



##Define the architecture of the Convolutional Neural Network:


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model

def baseline_model():
    inputs = Input(shape=(28, 28, 1))

    x = Conv2D(32, (5, 5), activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu', name='f')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Instantiate the model
model = baseline_model()

# Train the model
model.fit(X_train1, y_train1, validation_data=(X_test1, y_test1), epochs=1, batch_size=200, verbose=2)

# Evaluate the model
scores = model.evaluate(X_test1, y_test1, verbose=0)
print("Accuracy of the model: %.2f%%" % (100 * scores[1]))




300/300 - 5s - 16ms/step - accuracy: 0.9315 - loss: 0.2349 - val_accuracy: 0.9744 - val_loss: 0.0824
Accuracy of the model: 97.44%


## Exercise 2: Use the trained CNN as Feature Extractor and classify images using a SVM

Define a new model by setting the output of the previous CNN just before the classification layers.  

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define input tensor
inputs = Input(shape=(28, 28, 1))

# Build the CNN architecture
x = Conv2D(32, (5, 5), activation='relu')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
features = Dense(128, activation='relu', name='f')(x)
outputs = Dense(num_classes, activation='softmax')(features)

# Create the model
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train1, y_train1, validation_data=(X_test1, y_test1), epochs=1, batch_size=200, verbose=2)

# Create feature extractor model (from input to 'f' layer)
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('f').output)

# Extract features for SVM
X_train_features = feature_extractor.predict(X_train1)
X_test_features = feature_extractor.predict(X_test1)

# Train and evaluate SVM
svm = SVC(kernel='linear')
svm.fit(X_train_features, y_train)
y_pred = svm.predict(X_test_features)
accuracy = accuracy_score(y_test, y_pred)
print("SVM Accuracy on test set: %.2f%%" % (100 * accuracy))

300/300 - 4s - 15ms/step - accuracy: 0.9260 - loss: 0.2550 - val_accuracy: 0.9761 - val_loss: 0.0824
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
SVM Accuracy on test set: 98.34%
